In [ ]:
import os
import site
site.addsitedir(r'D:\Users\mickey\aoLab\code\hierarchical_lfads')
from dataset import EcogTensorDataset, MultiblockEcogTensorDataset, FilterData, create_n_block_w
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from utils import read_data, read_multiband_data

In [ ]:
full_record_path = r'D:\Users\mickey\Data\datasets\ecog\goose_wireless\gw_250_renorm'
filt_record_path = r'D:\Users\mickey\Data\datasets\ecog\goose_wireless\gw_250_nband2'
full_record = h5py.File(full_record_path,mode='r')
filt_record = h5py.File(filt_record_path,mode='r')
full_dict = {'test_ecog': full_record['test_ecog'][()]}
filt_dict = {'band0_test_ecog': filt_record['band0_test_ecog'][()],
             'band1_test_ecog': filt_record['band1_test_ecog'][()]}
n_block = 2
device = 'cuda'
# create h5 dataset
h5_ds = MultiblockEcogTensorDataset(full_record,filt_record,n_band=n_block,part_str='test',device=device)
# create dict dataset
dict_ds = MultiblockEcogTensorDataset(full_dict,filt_dict,n_band=n_block,part_str='test',device=device)

In [ ]:
n_index = 1000
t_h5 = []
t_dict = []
for idx in tqdm(range(n_index)):
    #   h5 dataset sampling
    t_h5_start = time.time()
    h5_sample = h5_ds.__getitem__(idx)
    t_h5_end = time.time()
    t_h5.append(t_h5_end - t_h5_start)
    #   dict dataset sampling
    t_dict_start = time.time()
    dict_sample = dict_ds.__getitem__(idx)
    t_dict_end = time.time()
    t_dict.append(t_dict_end-t_dict_start)
t_h5 = np.array(t_h5)
t_dict = np.array(t_dict)

In [ ]:
print(np.mean(t_h5),np.std(t_h5))
print(np.mean(t_dict),np.std(t_dict))
plt.hist(t_h5,100,label='h5')
plt.hist(t_dict,100,label='dict')
plt.legend(loc=0)

In [ ]:
print(f'{t_h5.mean():0.3e}\t{t_h5.var():0.3e}\t[{np.percentile(t_h5,[2.5,97.5])}]')
print(f'{t_dict.mean():0.3e}\t{t_dict.var():0.3e}\t[{np.percentile(t_dict,[2.5,97.5])}]')

SO - the dict loader is ~2x the speed of the record loader. If you CAN load all your training/validation data into CPU-side memory, do it. If you can't, it will take on average 2x the time to process each epoch with this implementation.